# End-to-end Sliceline application

_____________________________
This demo notebook is split in 2 parts:

- **Machine Learning modelling**

This part implements a basic classification pipeline on the [Titanic dataset](https://www.openml.org/search?type=data&sort=runs&id=40945&status=active) to predict if a passanger survived.

- **Model debugging with Sliceline**

This part identifies slices where the training error of the model is significantly higher, thanks to [sliceline](https://github.com/DataDome/sliceline).

## Machine Learning modelling

The pipeline is composed of 2 steps:
1. The preprocessor: to transform raw data into numerical data without NaN,
2. The classifier: a [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) with default parameters.

The training error is the element-wise [log loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html).

In [1]:
# import useful modules
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

# fetch titanic dataset
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X.drop(['cabin', 'boat', 'body', 'home.dest', 'name', 'ticket'], axis=1, inplace=True)

# define pipeline
cat_cols = X.select_dtypes("category").columns
num_cols = X.select_dtypes("number").columns

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
])

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols),
    ])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state=42))])

# training
clf.fit(X, y)

# predict on training data
y_proba = clf.predict_proba(X)[:, 1] # score of being a '1'

# compute element-wise log loss (the lower, the better)
eps = 1e-15
y_proba = np.clip(y_proba, eps, 1 - eps)
y = y.astype(int)

training_errors = - (y * np.log(y_proba) + (1 - y) * np.log(1 - y_proba))

## Model debbuging with Sliceline

**Sliceline considers all the columns of the input dataset as categorical.**

So, to get more relevant slices, the following numerical features should be discretized:
- `age`
- `fare`

Indeed, those columns as-is would lead to poor exploitable results. We would rather have range of values to specific value in our slices definition.

To discretize them and compute their bins, we use [OptBinning](http://gnpalencia.org/optbinning/) but feel free to experiment other binning implementations.

Sliceline configuration:
- `alpha = 0.95`: we are interested in small slice with high log loss.
- `k = 1`: we want Sliceline to find the rules with the best score.
- `max_l = X_trans.shape[1]`: we want Sliceline to be able to use all of the inputs columns.
- `min_sup = 1`: because the input dataset is relatively small, we do not add constraint regarding the minimal support.

In [2]:
# import sliceline and binning class
from sliceline.slicefinder import Slicefinder
from optbinning import ContinuousOptimalBinning

# dataset before prediction
X_trans = pd.DataFrame(clf[0].transform(X), columns=clf[0].get_feature_names_out())

# `age` and `fare` have to be bined
columns_to_bin = ["num__age", "num__fare"]

optimal_binner = ContinuousOptimalBinning()

X_trans[columns_to_bin] = np.array(
    [
        optimal_binner.fit_transform(X_trans[col], training_errors, metric="bins") for col in columns_to_bin
    ]
).T

# fitting sliceline
sf = Slicefinder(
    alpha = 0.95,
    k = 1,
    max_l = X_trans.shape[1],
    min_sup = 1,
    verbose = True
)

sf.fit(X_trans, training_errors)

(CVXPY) Mar 24 12:09:15 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.12.4544). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')


(CVXPY) Mar 24 12:09:15 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.12.4544). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')


/Users/antoinededaran/Library/Caches/pypoetry/virtualenvs/sliceline-Cft-FNPw-py3.10/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
DEBUG:sliceline.slicefinder:Dropping 0/40 features below min_sup = 1.


DEBUG:sliceline.slicefinder:Initial top-K: count=1, max=0.411321, min=0.411321


DEBUG:sliceline.slicefinder:Level 2:


...

DEBUG:sliceline.slicefinder: -- generated paired slice candidates: 525 -> 51


DEBUG:sliceline.slicefinder: -- valid slices after eval: 51/51


DEBUG:sliceline.slicefinder: -- top-K: count=40, max=0.914981, min=0.914981


DEBUG:sliceline.slicefinder:Terminated at level 10.


Slicefinder(alpha=0.95, max_l=10, min_sup=1)

In [3]:
# slices found
pd.DataFrame(sf.top_slices_, columns=sf.feature_names_in_, index=sf.get_feature_names_out())

,num__pclass,num__age,num__sibsp,num__parch,num__fare,cat__sex_female,cat__sex_male,cat__embarked_C,cat__embarked_Q,cat__embarked_S
slice_0,None,"[36.90, 39.20)",None,None,"(-inf, 7.89)",None,None,None,1.0,None
slice_1,None,"[36.90, 39.20)",None,0.0,"(-inf, 7.89)",None,None,0.0,None,0.0
slice_2,None,"[36.90, 39.20)",None,0.0,"(-inf, 7.89)",None,None,0.0,1.0,None
slice_3,None,"[36.90, 39.20)",0.0,None,"(-inf, 7.89)",None,None,None,1.0,0.0
slice_4,None,"[36.90, 39.20)",0.0,None,"(-inf, 7.89)",None,None,0.0,None,0.0
slice_5,None,"[36.90, 39.20)",0.0,None,"(-inf, 7.89)",None,None,0.0,1.0,None
slice_6,None,"[36.90, 39.20)",0.0,0.0,"(-inf, 7.89)",None,None,None,1.0,None
slice_7,None,"[36.90, 39.20)",None,None,"(-inf, 7.89)",None,None,0.0,1.0,0.0
slice_8,None,"[36.90, 39.20)",None,0.0,"(-inf, 7.89)",None,None,None,1.0,0.0
slice_9,3.0,"[36.90, 39.20)",None,None,"(-inf, 7.89)",None,None,None,1.0,0.0


**Note:**

We found 40 slices with `k` set to 1. As described in the documentation, it means that those 40 slices have the same score.

**In fact, they target the same subset of data.**

_(`None` values refer to unused features in each slices.)_

In [4]:
from sklearn.metrics import log_loss

# select one slice
slice_index = 0
current_slice = sf.top_slices_[slice_index]

# create a pandas filter
predicate_conditions = [X_trans[feature_name] == feature_value for feature_name, feature_value in zip(
    sf.feature_names_in_, current_slice) if feature_value is not None]
condition = " & ".join(
    [f"@predicate_conditions[{i}]" for i in range(len(predicate_conditions))]
)

# get slice element indices
indices = X_trans.query(condition).index

print("Model log loss on:")
print(f"- the full dataset ({X.shape[0]} passengers):", log_loss(y, y_proba))
print(f"- the selected slice ({len(indices)} passengers):", log_loss(y.iloc[indices], y_proba[indices]))

Model log loss on:
- the full dataset (1309 passengers): 0.14176053850706685
- the selected slice (33 passengers): 0.5667905434348764


# Conclusion

With Sliceline, we identified a subset of 33 passengers on which the model performs significantly worse. Those passengers:
- were in 3rd class (`num__pclass=3`),
- were between 37 and 39 years old (`num__age='[36.90, 39.20)'`),
- without any parents or children aboard (`num__parch=0.0`),
- and embarked in Queenstown (`cat__embarked_Q=1`).

To improve the modelisation, we should focus on reducing the error on those passengers.